# Heart Health Ensemble

In [177]:
import warnings
warnings.filterwarnings('ignore')

In [178]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read Data and Perform Data Cleaning(?)

In [179]:
columns = [
    "HeartDisease",
    "BMI",
    "Smoking",
    "AlcoholDrinking",
    "Stroke",
    # "PhysicalHealth",
    "MentalHealth",
    "DiffWalking",
    "Sex",
    "AgeCategory",
    "Race",
    "Diabetic",
    "PhysicalActivity",
    "SleepTime",
    "Asthma",
    "KidneyDisease",
    "SkinCancer"
]

target = ["health_status"]

In [180]:
# Load the data
file_path = Path('./DataTables/heart_2020_cleaned.csv')
df = pd.read_csv(file_path)
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()
df.head(10)

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,30,No,Female,55-59,White,Yes,Yes,5,Yes,No,Yes
1,No,20.34,No,No,Yes,0,No,Female,80 or older,White,No,Yes,7,No,No,No
2,No,26.58,Yes,No,No,30,No,Male,65-69,White,Yes,Yes,8,Yes,No,No
3,No,24.21,No,No,No,0,No,Female,75-79,White,No,No,6,No,No,Yes
4,No,23.71,No,No,No,0,Yes,Female,40-44,White,No,Yes,8,No,No,No
5,Yes,28.87,Yes,No,No,0,Yes,Female,75-79,Black,No,No,12,No,No,No
6,No,21.63,No,No,No,0,No,Female,70-74,White,No,Yes,4,Yes,No,Yes
7,No,31.64,Yes,No,No,0,Yes,Female,80 or older,White,Yes,No,9,Yes,No,No
8,No,26.45,No,No,No,0,No,Female,80 or older,White,"No, borderline diabetes",No,5,No,Yes,No
9,No,40.69,No,No,No,0,Yes,Male,65-69,White,No,Yes,10,No,No,No


In [181]:
# Define AgeCategory Bins dictionary
file_path = Path('./DataTables/AgeRecoded.csv')
ageDF = pd.read_csv(file_path)
ageDict = dict(zip(ageDF.AgeCategory, ageDF.AgeRisk))
print(ageDict)

{'18-24': 'Low Risk', '25-29': 'Low Risk', '30-34': 'Low Risk', '35-39': 'Low Risk', '40-44': 'Low Risk', '45-49': 'Medium Risk', '50-54': 'Medium Risk', '55-59': 'Medium Risk', '60-64': 'Medium Risk', '65-69': 'High Risk', '70-74': 'High Risk', '75-79': 'High Risk', '80 or older': 'High Risk'}


In [182]:
# Define Diabetic Bins dictionary
file_path = Path('./DataTables/DiabeticRecoded.csv')
diabeticDF = pd.read_csv(file_path)
diabeticDict = dict(zip(diabeticDF.Diabetic, diabeticDF['Diabetes Bin']))
print(diabeticDict)

{'No': 'No', 'Yes': 'Yes', 'No, borderline diabetes': 'No', 'Yes (during pregnancy)': 'No'}


In [183]:
# Define SleepTime Bins dictionary
file_path = Path('./DataTables/SleepRecoded.csv')
sleepDF = pd.read_csv(file_path)
sleepDict = dict(zip(sleepDF.SleepTime, sleepDF['Recommended Sleep']))
print(sleepDict)

{1: 'Below', 2: 'Below', 3: 'Below', 4: 'Below', 5: 'Below', 6: 'Below', 7: 'Meets', 8: 'Meets', 9: 'Meets', 10: 'Above', 11: 'Above', 12: 'Above', 13: 'Above', 14: 'Above', 15: 'Above', 16: 'Above', 17: 'Above', 18: 'Above', 19: 'Above', 20: 'Above', 21: 'Above', 22: 'Above', 23: 'Above', 24: 'Above'}


In [184]:
# Replace Age, Diabetic, SleepTime values w/ Bin values
df.AgeCategory = df.AgeCategory.replace(ageDict)
df.Diabetic = df.Diabetic.replace(diabeticDict)
df.SleepTime = df.SleepTime.replace(sleepDict)

df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,30,No,Female,Medium Risk,White,Yes,Yes,Below,Yes,No,Yes
1,No,20.34,No,No,Yes,0,No,Female,High Risk,White,No,Yes,Meets,No,No,No
2,No,26.58,Yes,No,No,30,No,Male,High Risk,White,Yes,Yes,Meets,Yes,No,No
3,No,24.21,No,No,No,0,No,Female,High Risk,White,No,No,Below,No,No,Yes
4,No,23.71,No,No,No,0,Yes,Female,Low Risk,White,No,Yes,Meets,No,No,No


In [185]:
# Apply LabelEncoder module to all columns containing string values
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
string_columns = [    
    "HeartDisease",
    "Smoking",
    "AlcoholDrinking",
    "Stroke",
    "DiffWalking",
    "Sex",
    "AgeCategory",
    "Race",
    "Diabetic",
    "PhysicalActivity",
    "SleepTime",
    "Asthma",
    "KidneyDisease",
    "SkinCancer"    ]
df2 = df.copy()
df2[string_columns] = df2[string_columns].apply(le.fit_transform)
df2.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,SleepTime,Asthma,KidneyDisease,SkinCancer
0,0,16.60,1,0,0,30,0,0,2,5,1,1,1,1,0,1
1,0,20.34,0,0,1,0,0,0,0,5,0,1,2,0,0,0
2,0,26.58,1,0,0,30,0,1,0,5,1,1,2,1,0,0
3,0,24.21,0,0,0,0,0,0,0,5,0,0,1,0,0,1
4,0,23.71,0,0,0,0,1,0,1,5,0,1,2,0,0,0


# Split Data into Training and Testing

In [126]:
# Create our features
X = df.drop("health_status", axis=1)
X = pd.get_dummies(X)

# Create our target
y = df.loc[:, target].copy()
X.describe()

KeyError: "['health_status'] not found in axis"

In [127]:
# Check the balance of our target values
y['health_status'].value_counts()

NameError: name 'y' is not defined

In [128]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)
print(Counter(y_train['health_status']))
print(Counter(y_test['health_status']))

NameError: name 'X' is not defined

In [129]:
features = clf.feature_importances_
print(features)
plt.bar(x = range(len(features)), height=features)
plt.show()

NameError: name 'clf' is not defined

In [130]:
from sklearn.feature_selection import SelectFromModel

In [131]:
from sklearn.feature_selection import SelectFromModel
sel = SelectFromModel(clf)
sel.fit(X_train_scaled, y_train)
sel.get_support()

NameError: name 'clf' is not defined

In [132]:
array([])

NameError: name 'array' is not defined

In [133]:
X_selected_train, X_selected_test, y_train, y_test = train_test_split(sel.transform(X), y, random_state=1)
scaler = StandardScaler().fit(X_selected_train)
X_selected_train_scaled = scaler.transform(X_selected_train)
X_selected_test_scaled = scaler.transform(X_selected_test)

NameError: name 'sel' is not defined

In [134]:
clf = LogisticRegression().fit(X_train_scaled, y_train)
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')  Out [8]:  

SyntaxError: invalid syntax (3846356844.py, line 3)

In [135]:
clf = LogisticRegression()
clf.fit(X_selected_train_scaled, y_train)
print(f'Training Score: {clf.score(X_selected_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_selected_test_scaled, y_test)}')

NameError: name 'LogisticRegression' is not defined

# Ensemble Learners
In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data.
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the imbalanced_classification_report from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score
Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Easy Ensemble AdaBoost Classifer

In [136]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
EE_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
EE_model.fit(X_train, y_train)
print(Counter(y_train['health_status']))

NameError: name 'X_train' is not defined

In [137]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test,y_pred)

NameError: name 'y_test' is not defined

In [138]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

NameError: name 'y_test' is not defined

In [139]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

NameError: name 'y_test' is not defined

### Naive Random Oversampling

In [140]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler (random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
print(Counter(y_resampled['health_status']))

NameError: name 'X_train' is not defined

In [141]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

NameError: name 'X_resampled' is not defined

In [142]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test,y_pred)

NameError: name 'y_test' is not defined

In [143]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

NameError: name 'X_test' is not defined

In [144]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

NameError: name 'y_test' is not defined

### SMOTE Oversampling

In [145]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train, y_train)
print(Counter(y_resampled['health_status']))

NameError: name 'X_train' is not defined

In [146]:
# Train the Logistic Regression model using the resampled data
modelS = LogisticRegression(solver='lbfgs', random_state=1)
modelS.fit(X_resampled, y_resampled)

NameError: name 'X_resampled' is not defined

In [147]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

NameError: name 'y_test' is not defined

In [148]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

NameError: name 'y_test' is not defined

## Undersampling
In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using Counter from the collections library.
2. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the imbalanced_classification_report from imbalanced-learn.
Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [149]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
print(Counter(y_resampled["health_status"]))

NameError: name 'X_train' is not defined

In [150]:
# Train the Logistic Regression model using the resampled data
modelU = LogisticRegression(solver='lbfgs', random_state=1)
modelU.fit(X_resampled, y_resampled)

NameError: name 'X_resampled' is not defined

In [151]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

NameError: name 'y_test' is not defined

In [152]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

NameError: name 'y_test' is not defined

## Combination (Over and Under) Sampling
In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using Counter from the collections library.
2. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the imbalanced_classification_report from imbalanced-learn.
Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [153]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X,y)
print(Counter(y_resampled["health_status"]))

NameError: name 'X' is not defined

In [154]:
# Train the Logistic Regression model using the resampled data
modelENN = LogisticRegression(solver='lbfgs',random_state=1)
modelENN.fit(X_resampled, y_resampled)

NameError: name 'X_resampled' is not defined

In [155]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

NameError: name 'y_test' is not defined

In [156]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

NameError: name 'y_test' is not defined

In [157]:
#Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

NameError: name 'y_test' is not defined